In [1]:
import pandas as pd

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
dataset_url = "https://raw.githubusercontent.com/Scherpinski-R/Brain-Stroke-Prediction/main/BrainStrokeData/full_data.csv"

In [4]:
df = pd.read_csv(dataset_url)

### Como sugerido pelo autor do Dataset, e como temos muitos mais dados para Stroke=0 do que Stroke=1, droparemos instancias com idade < 38

In [6]:
df = df.drop( df[df.age < 38].index )

In [7]:
y = df["stroke"]
X = df.loc[:, df.columns!="stroke"]

In [8]:
from imblearn.under_sampling import OneSidedSelection

In [10]:
categorical_features = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]

In [11]:
numerical_features = ["age", "avg_glucose_level", "bmi"]

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
columns_trans = ColumnTransformer(
    transformers=[
    ("cat", OneHotEncoder(), categorical_features),
    ("num", StandardScaler(), numerical_features)], 
    remainder='passthrough'
)

In [16]:
from imblearn.pipeline import Pipeline 

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Undersample com OneSidedSelection = TomekLink + Condensed Nearest Neighbor

https://machinelearningmastery.com/undersampling-algorithms-for-imbalanced-classification/

https://imbalanced-learn.org/stable/references/generated/imblearn.pipeline.Pipeline.html

https://tiaplagata.medium.com/how-scikit-learn-pipelines-make-your-life-so-much-easier-3cfbfa1d9da6

Foi necessario mudar pipeline pro imblearn pois nao ha suporte no pipeline do sklearn com OneSidedSelection()

In [44]:
pipe = Pipeline( [ ('columns_trans',columns_trans), ('oss', OneSidedSelection(n_neighbors=1, n_seeds_S=200)), ('knn', KNeighborsClassifier(n_neighbors=3)) ])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y) # TO-DO: Criar K-Fold 

In [45]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columns_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['gender', 'ever_married',
                                                   'work_type',
                                                   'Residence_type',
                                                   'smoking_status']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'avg_glucose_level',
                                                   'bmi'])])),
                ('oss', OneSidedSelection(n_neighbors=1, n_seeds_S=200)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [46]:
pipe.score(X_test, y_test) # To-DO: Criar matriz de confusao

0.8607427055702918

In [22]:
y.shape

(3015,)

In [23]:
y.sum()

245

In [49]:
y_train.to_numpy().sum()

180

In [50]:
y_train.shape

(2261,)

In [28]:
from sklearn.metrics import confusion_matrix

In [47]:
c_m = confusion_matrix(y_test, pipe.predict(X_test))
c_m

array([[644,  45],
       [ 60,   5]])

Em casos de Diagnosticos o objetivo eh minimizar os falsos positivos,
podemos trocar os modelos knn ou testas novos hiperparametros

obs.: possivel utilizar 3 conjuntos de dados (train, test, validate)

In [48]:
falsos_positivos = c_m[1][0] / c_m.sum()
print("Falso Positivo = {:.3f}%".format(falsos_positivos*100))

Falso Positivo = 7.958%
